In [9]:
%reload_ext autoreload
%autoreload 2

In [10]:
import exploring_script as es
import importlib
importlib.reload(es)
import pandas as pd
import numpy as np


from astropy.time import Time

In [51]:
confirmed_sso = es.load_data(columns=["ssnamenr", "jd", "nid", "ra", "dec", "magpsf", "sigmapsf"])

In [21]:
rocks_names = pd.read_parquet("data/rocks_fink.parquet")
mpc_database = pd.read_parquet(
    "../data/MPC_Database/mpcorb_extended.parquet",
    columns=["Number", "Name", "Principal_desig", "Other_desigs", "a", "e", "i", "Node", "Peri", "M", "Epoch", "Orbit_type"]
)

In [22]:
start_jd = Time("2020-09-01").jd
end_jd = Time("2020-10-01").jd

In [23]:
exp_paper_data = confirmed_sso[(confirmed_sso["jd"] > start_jd) & (confirmed_sso["jd"] < end_jd)]

In [24]:
mpc_in_fink, fink_not_in_mpc = es.mpc_crossmatch(mpc_database, pd.Series(exp_paper_data["ssnamenr"].unique()))

In [33]:
orbfit_limit_point=6
tw=15


is_detectable = (
        exp_paper_data.sort_values("jd")
        .groupby("ssnamenr")
        .agg(nb_det=("ra", len), is_in_tw=("jd", lambda x: np.all(np.diff(x) <= tw)))
        .reset_index()
    )

In [39]:
exp_paper_data = exp_paper_data.merge(is_detectable, on="ssnamenr")

In [42]:
exp_paper_data_detectable = exp_paper_data[(exp_paper_data["nb_det"] >= orbfit_limit_point) & (exp_paper_data["is_in_tw"])]

In [47]:
mpc_in_detectable, detectable_not_in_mpc = es.mpc_crossmatch(mpc_database, pd.Series(exp_paper_data_detectable["ssnamenr"].unique()))

In [49]:
mpc_in_fink["Orbit_type"].value_counts()

MBA                                           80594
Jupiter Trojan                                 2065
Hungaria                                       1596
Phocaea                                        1127
Object with perihelion distance < 1.665 AU      695
Hilda                                           349
Amor                                            111
Apollo                                           87
Distant Object                                   29
Aten                                             13
Name: Orbit_type, dtype: int64

In [48]:
mpc_in_detectable["Orbit_type"].value_counts()

MBA                                           42237
Jupiter Trojan                                 1058
Hungaria                                        788
Phocaea                                         635
Object with perihelion distance < 1.665 AU      407
Hilda                                           189
Amor                                             53
Apollo                                           49
Distant Object                                   12
Aten                                              5
Name: Orbit_type, dtype: int64

In [52]:
mpc_in_detectable

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type
33,(34),Circe,A855 GA,1965 JL,2.686888,0.106587,5.49740,184.33198,330.04966,80.66508,2460000.5,MBA
44,(45),Eugenia,A857 MA,1941 BN,2.720880,0.083354,6.60508,147.59158,87.54349,346.55692,2460000.5,MBA
50,(51),Nemausa,A858 BA,1949 HC1,2.364919,0.067404,9.97939,175.95499,1.69084,116.27059,2460000.5,MBA
61,(62),Erato,A860 RD,A906 BE,3.130603,0.168109,2.23647,125.12030,277.48090,127.94379,2460000.5,MBA
66,(67),Asia,A861 HA,None,2.422194,0.184575,6.02929,202.38775,107.06734,284.59770,2460000.5,MBA
...,...,...,...,...,...,...,...,...,...,...,...,...
1088217,None,None,2017 YG13,2015 EU4,2.289755,0.234933,4.04789,53.75587,330.70343,232.09054,2460000.5,MBA
1091374,None,None,2018 BA18,None,2.732153,0.236757,8.51407,93.65058,250.96625,205.93107,2460000.5,MBA
1111028,None,None,2019 AS55,None,1.932107,0.113588,17.87981,303.87141,56.44723,318.19537,2460000.5,Hungaria
1115074,None,None,2019 GP45,None,2.583391,0.288735,10.49456,16.31149,329.24792,223.04968,2460000.5,MBA


In [53]:
bft_data = pd.read_parquet("../data/BFT_Miriade/ssoBFT-latest.parquet", 
                columns=["sso_id", "sso_number", "sso_name", "sso_type", "sso_class"])

In [67]:
rocks_names["ast_number"] = rocks_names["ast_number"].fillna(-1).astype(int).astype(str)

In [92]:
def get_label(x):
        if x[0] == "MB":
            if x[1] in ["Middle", "Outer", "Inner"]:
                return x[0]
            else:
                return x[1]
        elif x[0] == "NEA":
            return x[1]
        else:
            return x[0]

def merge_bft(data):
    merge = (data
            .merge(rocks_names, left_on="ssnamenr", right_on="ast_number")
            .merge(bft_data, left_on="ast_name", right_on="sso_name")
            )
    merge["class_alt"] = (
        merge["sso_class"].str.split(">").map(lambda x: get_label(x))
    )

    return merge

In [93]:
exp_with_bft = merge_bft(exp_paper_data)

In [94]:
exp_with_bft["class_alt"].value_counts()

MB              715549
Trojan           16720
Hungaria         12443
Phocaea           8510
Mars-Crosser      7814
Cybele            3271
Hilda             3207
Apollo             800
Amor               736
KBO                116
Aten                85
Centaur             50
Name: class_alt, dtype: int64

In [97]:
kbo_class = exp_with_bft[exp_with_bft["class_alt"] == "KBO"]

In [98]:
Time(kbo_class.sort_values("jd").groupby("ssnamenr").agg(
    first_jd=("jd", lambda x: list(x)[0])
)["first_jd"], format="jd").iso

array(['2020-09-22 11:01:55.998', '2020-09-04 09:36:18.999',
       '2020-09-18 11:20:42.999', '2020-09-04 08:52:21.000',
       '2020-09-16 11:39:19.996', '2020-09-20 08:47:22.998',
       '2020-09-16 11:38:40.001', '2020-09-28 12:17:31.998',
       '2020-09-15 09:29:32.997', '2020-09-22 09:39:26.004',
       '2020-09-22 11:26:36.004', '2020-09-04 09:04:47.004',
       '2020-09-19 10:09:41.000', '2020-09-15 10:08:14.004',
       '2020-09-22 06:39:48.001', '2020-09-20 04:20:56.000',
       '2020-09-19 10:03:37.999', '2020-09-24 09:04:46.001',
       '2020-09-04 08:09:17.001', '2020-09-27 11:01:53.000',
       '2020-09-26 06:49:53.000', '2020-09-16 11:48:47.998'], dtype='<U23')

In [103]:
first_night = pd.read_parquet("../../kbo_neo_issue_ff_output/mpc/trajectory_df.parquet")

In [107]:
bft_first_night = merge_bft(first_night)
bft_first_night["class_alt"].value_counts()

MB              47029
Phocaea           734
Mars-Crosser      664
Hungaria          642
Trojan            526
Cybele            257
Hilda             197
Apollo             74
Amor               53
Aten                4
KBO                 2
Name: class_alt, dtype: int64

In [112]:
bft_first_night[bft_first_night["class_alt"] == "KBO"]["trajectory_id"]

50071    24532
50082    24638
Name: trajectory_id, dtype: int64

In [114]:
bft_first_night[bft_first_night["trajectory_id"] == 24532]

,ra,dec,jd,nid,fid,magpsf,sigmapsf,candid,not_updated,last_assoc_date,...,ssnamenr,assoc_tag,ast_name,ast_number,sso_id,sso_number,sso_name,sso_type,sso_class,class_alt
29576,26.744931,-1.229936,2.459095e+06,1340,1,18.094423,0.094016,1340498484515015021,True,2020-09-02,...,10546,N,Nakanomakoto,10546,Nakanomakoto,10546.0,Nakanomakoto,Asteroid,MB>Inner,MB
50071,26.591834,-1.549407,2.459097e+06,1342,2,18.594046,0.088013,1342400214515015018,True,2020-09-04,...,136199,N,Eris,136199,Eris,136199.0,Eris,Dwarf Planet,KBO>Detached,KBO
